Цей код створює 2 окремі датафрейми:
- df_duration_ouliers - містить аутлаєри по часу поїздки > 24 годин.
- df - містить сирі дані очищені від аутлаєрів > 24 годин.

В обох датафреймах міститься ще 2 додаткові колонки: started_at_dt і ended_at_dt, що містять час початку і кінця поїздки в типі datetime в таймзомі міста Осло.

In [5]:
import pandas as pd
import datetime
import time
import helpful_functions
import pytz

In [6]:
city_name = 'bergen'        # можливі варіанти: olso, bergen, trondheim

start_month = 6           # 1 - 12
start_year = 2018         # 2019 - 2022

end_month = 6             # 1 - 12
end_year = 2022           # 2019 - 2022

data_folder = './data'

In [7]:
dates_str_list = helpful_functions.generate_months_in_period(
    start_month=start_month, 
    start_year=start_year, 
    end_month=end_month, 
    end_year=end_year, 
    sep="_")

In [8]:
start = time.time()
df = pd.read_csv(f"{data_folder}/{city_name}/{city_name}_{dates_str_list[0]}.csv")
for date in dates_str_list[1:]:
    try:
        df = pd.concat([df, pd.read_csv(f"{data_folder}/{city_name}/{city_name}_{date}.csv")])
    except:
        pass
end = time.time()
print("Зчитування: ", end - start, "с. \n")
print("Розмір початкової таблиці: ", df.shape)

Зчитування:  21.169935941696167 с. 

Розмір початкової таблиці:  (3035172, 13)


In [12]:
df[(df['duration'] > 6 * 60 * 60) & (df['duration'] <= 24 * 60 * 60)].shape

(715, 13)

In [5]:
strptime_pattern = "%Y-%m-%d %H:%M:%S%z"
timezone = pytz.timezone('Europe/Oslo')

start = time.time()
df['started_at_dt'] = df['started_at'].apply(lambda x: datetime.datetime.strptime(x[:-6].split('.')[0] + x[-6:], strptime_pattern).astimezone(timezone))
df['ended_at_dt'] = df['ended_at'].apply(lambda x: datetime.datetime.strptime(x[:-6].split('.')[0] + x[-6:], strptime_pattern).astimezone(timezone))
end = time.time()
print("Трансформування: ", end - start, "с.")
df.head()

Трансформування:  0.03181004524230957 с.


,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude,started_at_dt,ended_at_dt
0,2018-06-29 10:45:12.736000+00:00,2018-06-29 11:05:10.430000+00:00,1197,3,Grieghallen,,60.388167,5.328335,83,Bergen jernbanestasjon,,60.390324,5.332440,2018-06-29 12:45:12+02:00,2018-06-29 13:05:10+02:00
1,2018-06-29 10:53:59.691000+00:00,2018-06-29 11:11:28.184000+00:00,1048,49,Studentsenteret UIB,,60.387198,5.322980,75,Akvariet,,60.399877,5.304686,2018-06-29 12:53:59+02:00,2018-06-29 13:11:28+02:00
2,2018-06-29 11:01:02.987000+00:00,2018-06-29 11:09:13.172000+00:00,490,58,Tårnplass,,60.393755,5.321792,157,Høyteknologisenteret,Høyteknologisenteret,60.382255,5.332332,2018-06-29 13:01:02+02:00,2018-06-29 13:09:13+02:00
3,2018-06-29 11:30:33.362000+00:00,2018-06-29 11:38:32.276000+00:00,478,157,Høyteknologisenteret,Høyteknologisenteret,60.382255,5.332332,83,Bergen jernbanestasjon,,60.390324,5.332440,2018-06-29 13:30:33+02:00,2018-06-29 13:38:32+02:00
4,2018-06-29 11:40:44.376000+00:00,2018-06-29 12:10:23.766000+00:00,1779,58,Tårnplass,,60.393755,5.321792,116,Lysverket,"Gamle lysverket, ved KODE",60.389543,5.329622,2018-06-29 13:40:44+02:00,2018-06-29 14:10:23+02:00


In [11]:
df[['started_at', 'ended_at', 'started_at_dt', 'ended_at_dt']].head()

,started_at,ended_at,started_at_dt,ended_at_dt
0,2018-06-29 10:45:12.736000+00:00,2018-06-29 11:05:10.430000+00:00,2018-06-29 12:45:12+02:00,2018-06-29 13:05:10+02:00
1,2018-06-29 10:53:59.691000+00:00,2018-06-29 11:11:28.184000+00:00,2018-06-29 12:53:59+02:00,2018-06-29 13:11:28+02:00
2,2018-06-29 11:01:02.987000+00:00,2018-06-29 11:09:13.172000+00:00,2018-06-29 13:01:02+02:00,2018-06-29 13:09:13+02:00
3,2018-06-29 11:30:33.362000+00:00,2018-06-29 11:38:32.276000+00:00,2018-06-29 13:30:33+02:00,2018-06-29 13:38:32+02:00
4,2018-06-29 11:40:44.376000+00:00,2018-06-29 12:10:23.766000+00:00,2018-06-29 13:40:44+02:00,2018-06-29 14:10:23+02:00


In [15]:
df_start_stations = df[['start_station_id',
                        'start_station_name',
                           'start_station_description', 
                           'start_station_latitude', 
                           'start_station_longitude']]

In [28]:
a = df_start_stations.drop_duplicates(subset=['start_station_id', 'start_station_name',
                           'start_station_description', 
                           'start_station_latitude', 
                           'start_station_longitude'])

In [32]:
a.to_excel('1.xlsx', index=False)

In [36]:
a[a['start_station_id'].isin([449,451])].sort_values(by='start_station_id')

,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude
444,449,Ruseløkkgården,langs Løkkeveien,59.913499,10.72656
276,449,Ruseløkkgården,langs Løkkeveien,59.913575,10.72623
2000,451,Suhms gate,mellom Schultz' gate og Industrigata,59.929419,10.726195
30,451,Suhms gate,ved Oslo døvekirke,59.929419,10.726195
36678,451,Suhms gate,NaN,59.929419,10.726195


In [37]:
df.sort_values(by='duration').tail(1)

,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
107381,2020-06-12 22:58:29.631000+00:00,2020-09-21 11:40:40.126000+00:00,8685730,424,Birkelunden,langs Seilduksgata,59.925611,10.760926,489,Torggata,ved Bernt Ankers gate,59.915983,10.751551


In [38]:
8685730 / 60 / 60 / 24

100.5292824074074

In [75]:
df_start_stations = df_start_stations.drop_duplicates(subset=['start_station_id'],
                                                     keep='last')

In [76]:
df_start_stations.shape

(115, 5)

In [77]:
df_start_stations = df_start_stations.rename(columns={'start_station_id': 'station_id',
                                                     'start_station_name': 'station_name',
                                                     'start_station_description': 'station_description',
                                                     'start_station_latitude': 'station_latitude',
                                                     'start_station_longitude': 'station_longitude'})

In [78]:
df_start_stations.to_csv(f'./data/{city_name}_stations.csv', index=False)

In [79]:
df = df[['started_at_dt', 'ended_at_dt', 'duration', 'start_station_id', 'end_station_id']]

In [80]:
df.shape

(3035172, 5)

Створюємо 2 датафрейми: 
- df_duration_ouliers - містить аутлаєри по часу поїздки > 24 годин. Потрібний для подальшого аналізу цих аутлаєрів.
- df - містить сирі дані очищені від аутлаєрів > 24 годин. Для подальшого аналізу.

In [81]:
df_duration_outliers = df[df['duration'] > 24 * 60 * 60]
df = df[df['duration'] <= 24 * 60 * 60]

In [82]:
start = time.time()
df.to_csv(f'./data/{city_name}_clear_rides_data.csv', index=False)
end = time.time()
end - start

68.05525207519531

In [83]:
start = time.time()
df_duration_outliers.to_csv(f'./data/{city_name}_duration_outliers-stolen_bikes.csv', index=False)
end = time.time()
end - start

0.016646862030029297